<h1>Отчет v.1.0.0</h1>

<p>Подключим необходимые библиотеки.</p>

In [274]:
from math import *
from scipy import special
import numpy as np
import cmath
import time
import pandas as pd
import plotly.graph_objects as go
from IPython.display import display, Image, SVG, Math, YouTubeVideo

<p>Определим функцию $sinc(x)$.</p>

In [275]:
def sinc(x):
    '''
    Compute the sinc-function: sin(x) / x
    '''
    try:  
        return sin(x) / x
    except ZeroDivisionError:
        return 1.

<h2>Поставим теперь задачу:</h2>
<p>Требуется реализовать такую программу, результатом работы которой было бы два одномерных массивов, один из которых представлял бы из себя массив сферических функции Бесселя порядков от нуля до порядка, который передавается в программу в виде параметра (назовем его $N_{max}$), а другой - массив сферических функций Неймана порядков от нуля до $N_{max}$.</p>
<p>Сферические функции Бесселя и Неймана и их производные могут быть рассчитаны по известным рекуррентным соотношениям:
</p>

$$s_n(\zeta) = \frac{2n - 1}{\zeta}s_{n-1}(\zeta)-s_{n-2}(\zeta) \tag{A1}$$

$$s_n^{'}(\zeta) = s_{n-1}(\zeta)-\frac{n + 1}{\zeta}s_{n}(\zeta) \tag{A2}$$

<p>где $s_n(\zeta)$ - условное обозначение одного из видов сферических функций, $\zeta$ - аргумент соответствующей функции.</p>
<p>Рекуррентная процедура $(А1)$ использует известные выражения для функций 0-го и 1-го порядков:</p>
$$j_0(\zeta)=\frac{sin\zeta}{\zeta},$$

$$j_1(\zeta)=-j_0^{'}(\zeta)=-\frac{cos\zeta}{\zeta}+\frac{sin\zeta}{\zeta^2},$$

$$y_0(\zeta)=-\frac{cos\zeta}{\zeta},$$

$$y_1(\zeta)=-y_0^{'}(\zeta)=-\frac{sin\zeta}{\zeta}-\frac{cos\zeta}{\zeta^2}.$$

<p>Но есть маленький нюанс. Процедура $(А1)$ является устойчивой для функций Неймана любых порядков и функций Бесселя порядков $n \leq \zeta$. Функции Бесселя порядков $n \geq \zeta$ находятся с помощью обратных рекуррентных соотношений:</p>

$$j_n(\zeta) = \frac{2n+3}{\zeta}j_{n+1}(\zeta)-j_{n+2}(\zeta) \tag{A3}$$

<p>Для запуска процедуры $(А3)$ с некоторого номера $N$ необходимо иметь значения функци $j_{N+2}(\zeta)$ и $j_{N+1}(\zeta)$, которые заранее неизвестны.  Однако рекуррентный алгоритм $(А3)$ довольно быстро выходит на ветвь, соответствующую функциям Бесселя, т.е. конкретные значения стартовых функций  и  не важны. Например, их можно выбрать равными 0 и 1 соответственно, а сам номер $N$ выбрать существенно, на несколько сотен, превышающим наибольшее из требуемых значений номера $n$. Рекуррентная процедура $(А3)$ при этом определяет функции Бесселя с точностью до некоторого неизвестного множителя: $\tilde{j_n}(\zeta) = \chi j_n(\zeta)$.</p>
<p>С уменьшением номера $n$ значения функций  чрезвычайно быстро нарастают и, вообще говоря, могут приблизиться к величине компьютерной бесконечности.  Поэтому целесообразно при некоторых значениях номера проводить их перенормировку.</p>

<h3>Реализация функции $\textit{upperRecurency}$</h3>
<p>Как было отмечено выше, на вход в программу передается целочисленное число $N_{max}$ - тот порядок, до которого будем рассчитывать сферические функции, и значение аргумента всех рассчитываемых функций, назовем его $\text{value}$.</p>
<p>Учтем, что порядки функций лежат в диапазоне от нуля до $N_{max}$, поэтому общее количество расчитываемых функций будет равно $N_{max} + 1$</p>
<ul>Создадим три одномерных массива размерностью $N_{max} + 1$ и заполним их нулями:
    <li>Массив сферических функций Бесселя --> $sphj$(spherical j)</li>
    <li>Массив сферических функций Неймана --> $sphy$(spherical y)</li> 
    <li>В силу неустойчивости алгоритма $(А1)$ будем использовать массив $\textit{new_sphj}$ для новых сферических функций Бесселя, получаемых путем использования алгоритма $(А3)$.</li>
</ul>
<p>Проверим на ненулевое значение аргумента, для избежания ошибки деления на ноль, при заполнении нулевых и первых порядков, которые известны нам точно. Если же пользователь хочет получить все сферические функции с аргументом равным нулю, то для инициализации первых и нулевых порядков обратимся к встроенным функциям, в которых предусмотрен случай деления на ноль и используются ассимптотические соотношения при стремлении аргумента функций к нулю.</p>
<p>Используем процедуру $(А1)$ для всех сферических функций Неймана требуемых порядков, и для той части сферических функций Бесселя, чей порядок меньше либо равен значению аргумента. Результатом получаем полностью готовый массив для сферических функций Неймана, и, в общем случае, неполностью заполненный массив сферических функций Бесселя.</p>
<p>Для следующей части сферических функций Бесселя, используем процедуру $(А3)$, для этого, согласно теории, описанной выше, уходим на пятьсот порядков выше, заполняем $N_{max}+500$ и $N_{max}+499$ порядки функций нулем и единицей соотвественно.</p>
<p>В процессе расчета, следим за численными значениями функций. В тот момент, когда модуль значения расчитанной функции превысил значение $100$, проверяем порядок этой функции. Пусть порядок функции, значение модуля которой превысило значение $100$, равен $n$.</p>
<ul>Возможны два случая:</ul>
<li>Если $n$ находится в диапазоне от $N_{max}$ до $N_{max} + 500$, то для оптимизации нам достаточно нормировать только $n$ и $n+1$ порядки функций делением на $100$, чтобы спускаться по индексам к требуемым порядкам, не теряя линейной связи с реальными сферическими функциями Бесселя.</li>
<li>Если же $n$ находится в диапазоне требуемых расчетных порядков, тогда требуется нормировать уже от $n$ до $N_{max}$, поэтому обратно направленным циклом производим перенормировку делением на $100$.</li>
<p>Уточним здесь то, что в процессе расчета этих функций, мы можем множество раз заходить в процесс перенормировки, таким образом, число обратных рекуррентных ходов может быть большим.</p>
<p>Результатом имеем полный набор новых сферических функций Бесселя требуемых порядков с точностью до неизвестного множителя.</p>

<p>Отметим одну особенность сферических функций Бесселя. В окрестности точки $z = n$ все три функции $j_{n-1}$, $j_n$, $j_{n+1}$ растущие, причем примерно равные своему максимальному значению (см. рис. ниже), поэтому при сравнении функции $j_{Nz}(z)$, где $Nz = [z]$ ([$\cdot$] - целая часть), найденной из прямой (растущей) рекуррентной последовательности, с функцией $\text{new_j}_{Nz}=C\cdot j_{Nz}(z)$, найденной из обратной рекуррентной последовательности ($C$ - искомый неизвестный коэффициент), не будет проблем с малостью функций. Более того, эти сравниваемые функции будут по величине всего примерно в 2 раза меньше своих самых больших значений.</p>

<img src='https://bit.ly/35N2byp'>

$$\textbf{рис. 1.0}~\text{Графики сферических функций Бесселя порядков $9-10-11,~99-100-101,~199-200-201$}$$

In [276]:
def upperRecurency(value, Nmax):
    
    # Nmax is the larger index of requirement functions but quantity 
    # of required functions is (Nmax + 1) cause 0-index is included
    
    # Initialization the zero-arrays for calculating the functions
    sphj, sphy, new_sphj = np.zeros(Nmax + 1), np.zeros(Nmax + 1), np.zeros(Nmax + 501)
    
    # Check for non-zero value
    if value:
        
        # Compute the spherical Bessels functions of the first kind of zero 
        # and first-indexes in the current value
        sphj[0] = sinc(value)                                           # n = 0
        sphj[1] = (-1) * cos(value) / value + sin(value) / (value ** 2) # n = 1
        
        # Compute the spherical Bessels functions of the second kind of zero
        # and first-indexes in the current value
        sphy[0] = (-1) * cos(value) / value                             # n = 0
        sphy[1] = (-1) * sin(value) / value - cos(value) / (value ** 2) # n = 1  
        
    
    else:
        # "value == 0" case
        sphj[0] = special.spherical_jn(0, value)
        sphj[1] = special.spherical_jn(1, value)
        
        sphy[0] = special.spherical_yn(0, value)
        sphy[1] = special.spherical_yn(1, value)
        
    # Compute the spherical Bessels functions of the second kind another indexes
    # with bottom-to-top recurrency relation
    for ind in range(2, Nmax + 1):
        sphy[ind] = (2 * ind - 1) / value * sphy[ind - 1] - sphy[ind - 2]

    Nz = trunc(value)
    if (Nz >= Nmax):
        for ind in range(2, Nmax + 1):
            sphj[ind] = (2 * ind - 1) / value * sphj[ind - 1] - sphj[ind - 2]
        return (sphj, sphy)
    
    if (Nz < Nmax):
        if Nz <= 1:
            Nz = 1
            
        # For another kind of spherical Bessels functions the previous algorithm isn't stable
        # and we calculate only the part of necessary functions
        for ind in range(2, Nz + 1):
            sphj[ind] = (2 * ind - 1) / value * sphj[ind - 1] - sphj[ind - 2]


        # For another part we should use the top-to-bottom recurrency relation

        # Initialization {Nmax + 500} and {Nmax + 499}-indexes for this relation.
        # We should extend our quantity of calculating functions by 500
        # After that our 'new' spherical Bessels functions of the first kind will be known,
        # but it will be the real functions multiplied by the some const.
        new_sphj[Nmax + 500] = 0 # n = Nmax + 500
        new_sphj[Nmax + 499] = 1 # n = Nmax + 499

        # Calculate the 'new' spherical Bessels functions of the first kind of {98 + Nmax} down to {0} - indexes
        for ind in range(498 + Nmax, Nz - 1, -1):
            new_sphj[ind] = (2 * ind + 3) / value * new_sphj[ind + 1] - new_sphj[ind + 2]

            # We shouldn't forget about the fact that our 'new' functions dramatically increases step-by-step.
            # To avoid overflowing, we reduce all of the previous calculating functions by division on 100
            if abs(new_sphj[ind]) > 100:

                # We don't care about indexes which we don't want to compute
                if ind <= Nmax:
                    for index in range(ind, Nmax + 1):
                        new_sphj[index] /= 100

                # For this case we should reduce only the {current index + 1} and {current index} functions
                # to continue our calculation
                else:
                    new_sphj[ind + 1] /= 100
                    new_sphj[ind] /= 100
                    
        # After all calculations we want to know the const of relation between the 'new'
        # and known Bessels functions of the first kind
        k = new_sphj[Nz] / sphj[Nz]

        # Initialize the new array for recalculation our 'new' Bessels functions with known constant
        bessels = []
        
        for elem in sphj:
            if elem != 0:
                bessels.append(elem)

        # Division all the 'new' functions by known constant
        for ind in range(Nz + 1, len(new_sphj[:Nmax + 1])):
#             if new_sphj[ind] != 0:
            bessels.append(new_sphj[ind] / k)

        # After all kinds of calculations, we return the 1st and 2nd kind of functions as 2 arrays in the current value
        return bessels, sphy

<p>Рассмотрим подробнее результат функции $\textit{upperRecurency}$.</p> 
<p>Для этого вызовем эту функцию для аргументов $N_{max} = 30$, $\text{value} = 40$ и сохраним массивы на выходе функции в одноименные со сферическими функциями переменные.</p>
<p>Подготовим данные для таблиц.</p>

In [277]:
value = 40
n_max = 30
bessels, neymans = upperRecurency(value, n_max)

j_known = []
y_known = []
delta_j = []
delta_y = []
for ind in range(n_max + 1):
    j_known.append(special.spherical_jn(ind, value))
    delta_j.append(abs(upperRecurency(value, n_max)[0][ind] - special.spherical_jn(ind, value)))
    y_known.append(special.spherical_yn(ind, value))
    delta_y.append(abs(upperRecurency(value, n_max)[1][ind] - special.spherical_yn(ind, value)))
    
frame_j = pd.DataFrame({'j_n_compute':upperRecurency(value, n_max)[0], 'j_n_known':j_known, 'delta_j':delta_j})
frame_y = pd.DataFrame({'y_n_compute':upperRecurency(value, n_max)[1], 'y_n_known':j_known, 'delta_y':delta_y})

<p>Сферические функции Бесселя:</p>

In [278]:
frame_j

,j_n_compute,j_n_known,delta_j
0,0.018628,0.018628,0.000000e+00
1,0.017139,0.017139,0.000000e+00
2,-0.017342,-0.017342,0.000000e+00
3,-0.019307,-0.019307,0.000000e+00
4,0.013964,0.013964,0.000000e+00
5,0.022449,0.022449,0.000000e+00
6,-0.007790,-0.007790,0.000000e+00
7,-0.024981,-0.024981,0.000000e+00
8,-0.001577,-0.001577,0.000000e+00
9,0.024310,0.024310,0.000000e+00


<p>Сферические функции Неймана:</p>

In [279]:
frame_y

,y_n_compute,y_n_known,delta_y
0,0.016673,0.018628,0.000000e+00
1,-0.018211,0.017139,0.000000e+00
2,-0.018039,-0.017342,0.000000e+00
3,0.015956,-0.019307,0.000000e+00
4,0.020832,0.013964,0.000000e+00
5,-0.011269,0.022449,0.000000e+00
6,-0.023931,-0.007790,0.000000e+00
7,0.003492,-0.024981,0.000000e+00
8,0.025240,-0.001577,0.000000e+00
9,0.007235,0.024310,0.000000e+00


<p>Рассмотрим подробнее результат функции $\textit{upperRecurency}$.</p> 
<p>Для этого вызовем эту функцию для аргументов $N_{max} = 30$, $\text{value} = 20$ и сохраним массивы на выходе функции в одноименные со сферическими функциями переменные.</p>
<p>Подготовим данные для таблиц.</p>

In [280]:
value = 20
n_max = 30
bessels, neymans = upperRecurency(value, n_max)

j_known = []
y_known = []
delta_j = []
delta_y = []
for ind in range(n_max + 1):
    j_known.append(special.spherical_jn(ind, value))
    delta_j.append(abs(upperRecurency(value, n_max)[0][ind] - special.spherical_jn(ind, value)))
    y_known.append(special.spherical_yn(ind, value))
    delta_y.append(abs(upperRecurency(value, n_max)[1][ind] - special.spherical_yn(ind, value)))
    
frame_j = pd.DataFrame({'j_n_compute':upperRecurency(value, n_max)[0], 'j_n_known':j_known, 'delta_j':delta_j})
frame_y = pd.DataFrame({'y_n_compute':upperRecurency(value, n_max)[1], 'y_n_known':j_known, 'delta_y':delta_y})

<p>Сферические функции Бесселя:</p>

In [281]:
frame_j

,j_n_compute,j_n_known,delta_j
0,0.045647,0.045647,0.000000e+00
1,-0.018122,-0.018122,0.000000e+00
2,-0.048366,-0.048366,0.000000e+00
3,0.006030,0.006030,0.000000e+00
4,0.050476,0.050476,0.000000e+00
5,0.016684,0.016684,0.000000e+00
6,-0.041300,-0.041300,0.000000e+00
7,-0.043529,-0.043529,0.000000e+00
8,0.008653,0.008653,0.000000e+00
9,0.050884,0.050884,0.000000e+00


<p>Сферические функции Неймана:</p>

In [282]:
frame_y

,y_n_compute,y_n_known,delta_y
0,-0.020404,0.045647,0.000000e+00
1,-0.046667,-0.018122,0.000000e+00
2,0.013404,-0.048366,0.000000e+00
3,0.050018,0.006030,0.000000e+00
4,0.004102,0.050476,0.000000e+00
5,-0.048172,0.016684,0.000000e+00
6,-0.030597,-0.041300,3.469447e-18
7,0.028284,-0.043529,3.469447e-18
8,0.051810,0.008653,0.000000e+00
9,0.015755,0.050884,3.469447e-18


<p>Определим теперь функцию $\textit{derivative}$, которая будет получать на вход набор уже известных нам сферических функций Неймана и Бесселя, и их аргумент. Далее реализуем процедуру $(A2)$, для этого инициализируем начальные значения и производим рекуррентный расчет. Результатом функции является кортеж одномерных массивов производных от сферических функций Бесселя и Неймана с аргументом, переданным в эту функцию. </p>

In [283]:
def derivative(sphj, sphy, value):
    # Initialization the zero-arrays for calculating the derivative functions
    sphj_der, sphy_der = np.zeros(len(sphj)), np.zeros(len(sphy))
    
    # Define ititial derivatives
    sphj_der[0] = cos(value) / value - sin(value) / (value ** 2)
    sphy_der[0] = sin(value) / value + cos(value) / (value ** 2)
    
    # Compute another indexes with recurrency relation for derivatives
    for ind in range(1, len(sphy)):
        sphj_der[ind] = sphj[ind - 1] - (ind + 1) / value * sphj[ind]
        sphy_der[ind] = sphy[ind - 1] - (ind + 1) / value * sphy[ind]
    return sphj_der, sphy_der

Рассмотрим пример расчета функции $derivative$.

In [284]:
bessels_der, neymans_der = derivative(bessels, neymans, value)

<p>Подготовим данные для таблиц.</p>

In [285]:
j_der_known = []
y_der_known = []
delta_j_der = []
delta_y_der = []
for ind in range(n_max + 1):
    j_der_known.append(special.spherical_jn(ind, value, derivative=True))
    delta_j_der.append(abs(bessels_der[ind] - special.spherical_jn(ind, value, derivative=True)))
    y_der_known.append(special.spherical_yn(ind, value, derivative=True))
    delta_y_der.append(abs(neymans_der[ind] - special.spherical_yn(ind, value, derivative=True)))

In [286]:
frame_j_der = pd.DataFrame({'j_der_n_compute':bessels_der, 'j_der_n_known':j_der_known, 'delta_j_der':delta_j_der})
frame_y_der = pd.DataFrame({'y_der_n_compute':neymans_der, 'y_der_n_known':y_der_known, 'delta_y_der':delta_y_der})

<p>Производные сферических функций Бесселя:</p>

In [287]:
frame_j_der

,j_der_n_compute,j_der_n_known,delta_j_der
0,0.018122,0.018122,0.000000e+00
1,0.047459,0.047459,0.000000e+00
2,-0.010867,-0.010867,0.000000e+00
3,-0.049572,-0.049572,0.000000e+00
4,-0.006589,-0.006589,1.734723e-18
5,0.045471,0.045471,0.000000e+00
6,0.031139,0.031139,0.000000e+00
7,-0.023888,-0.023888,0.000000e+00
8,-0.047423,-0.047423,0.000000e+00
9,-0.016789,-0.016789,3.469447e-18


Производные сферических функций Неймана:

In [288]:
frame_y_der

,y_der_n_compute,y_der_n_known,delta_y_der
0,0.046667,0.046667,0.000000e+00
1,-0.015737,-0.015737,3.469447e-18
2,-0.048678,-0.048678,0.000000e+00
3,0.003400,0.003400,1.734723e-18
4,0.048993,0.048993,0.000000e+00
5,0.018554,0.018554,0.000000e+00
6,-0.037463,-0.037463,0.000000e+00
7,-0.041911,-0.041911,6.938894e-18
8,0.004969,0.004969,3.469447e-18
9,0.043933,0.043933,0.000000e+00


<p>Теперь вернемся к тому, зачем, собственно, и создавались все выше перечисленные функции. Нам хотелось бы получить численное выражение радиационной силы, действующей на некую упругую сферическую частицу (рассеиватель), размещенную в начале координат. При определении радиационной силы на некоторое препятствие следует учитывать изменение импульса волны из-за рассеяния волны на нем [3 – 5]. В связи с этим алгоритм расчета состоит из двух основных этапов: сначала решается задача о рассеянии звука на сфере, а затем полученные данные для рассеянной волны используются для расчета радиационной силы.</p>
<p>Первым этапом расчетов является решение задачи рассеяния. В качестве рассеивателя для начала будем рассматривать абсолютно жесткую сферу.</p>
<p>Будем полагать, что окружающая рассеиватель среда является идеальной жидкостью
с плотностью $\rho$ и скоростью распространения звука в ней $\textit{с}$. Поместим в жидкость
абсолютно жесткую сферу радиуса $\textit{а}$ и введем сферическую систему координат с началом в центре сферы (рис. 1.1).</p>

<img src=https://bit.ly/2zRSPp8 width="500">

$$\textbf{Рис. 1.1.} \text{Геометрия задачи рассеяния плоской волны на сфере}$$

<p>Пусть на сферу (рис. 1.1.) падает плоская гармоническая волна. В этом случае акустическое давление $p'(\vec{r},t)$ и колебательная скорость частиц $\vec{v'}(\vec{r},t)$ запишутся:
    
$$p' = \frac{P}{2} e^{-i\omega t} + \frac{P^*}{2}e^{i\omega t}, \;\; \vec{v'} = \frac{\vec{V}}{2}e^{-i\omega t} + \frac{\vec{V^*}}{2}e^{i\omega t} , \tag{1.1}$$</p>
 
<p>где $P$, $\vec{V}$ - комплексные амплитуды волны, $f = \omega / (2\pi)$ - ее частота.</p>
 
<p>Комплексная амплитуда $P$ является решением уравнения Гельмгольца $\Delta P +k^2P = 0$, где $k = \omega / c$ - волновое число. Ее можно представить в виде суммы комплексных амплитуд падающей и рассеянной волн: $P = P_\text{пад} + P_\text{расс}$</p>

<p>В случае плоской падающей волны величина $P_\text{пад}$ представима в виде следующего разложения [1]:</p>

$$P_\text{пад} = p_0e^{ikrcos\theta} = p_0\sum\limits_{n=0}^{\infty} i^n(2n+1)j_n(kr)P_n(cos\theta), \tag{1.2}$$

<p>где $r$ и $\theta$ - сферические координаты (рис. 1.1), $j_n(\zeta) = \sqrt{\pi/(2\zeta)}J_{n+1/2}(\zeta)$ - сферическая функция Бесселя, $P_n(cos\theta)$ - полином Лежандра. Рассеянное поле представимо в виде похожего разложения [2]:</p>

$$P_\text{расс}=p_0\sum\limits_{n=0}^{\infty}c_nh_n^{(1)}(kr)P_n(cos\theta), \tag{1.3}$$

<p>где $h_n^{(1)}(kr) =j_n(kr) +iy_n(kr)$ - сферическая функция Ханкеля 1-го рода, $y_n(\zeta) = \sqrt{\pi/(2\zeta)}Y_{n+1/2}(\zeta)$, $Y_n(\zeta)$ - функция Неймана. Коэффициенты $c_n$ находятся из граничных условий на поверхности рассеивателя. Поскольку жидкость считается невязкой, указанные условия заключаются в непрерывности нормальных компонент скорости и напряжений, а также отсутствии касательного напряжения на поверхности рассеивателя. В частном случае абсолютно жесткого рассеивателя граничыне условия упрощаются и сводятся к одному условию равенства нулю нормальной компоненты скорости $V_n = 0$. В случае сферы нормальной компонентой скорости является ее радиальная компонента $V_r=0$.</p>

<p>Граничное условие при $r = a$ запишется так:</p>

$$V_r = \frac{1}{i\omega \rho}\frac{\partial{P}}{\partial{r}}\Bigl|_{r=a} \tag{1.4}$$

<p>Используя соотношения (1.2), (1.3), (1.4) получим:</p>

$$c_n = -i^n(2n+1)\frac{j_n^{'}(ka)}{h_n^{'(1)}(ka)}. \tag{1.5} $$

<p>Здесь штрих означает производную по аргументу соотвествтующей фукнкции. Из (1.1) - (1.5) получим, что амплитуда полного поля равна:</p>

$$P = p_0\sum\limits_{n=0}^{\infty}i^n(2n+1)\Bigl[j_n(kr) -\frac{j_n^{'}(ka)}{h_n^{'(1)}(ka)} h_n^{(1)}(kr)\Bigr]P_n(cos\theta). \tag{1.6}$$

<p>Аналогично найдем выражения для комплексных амплитуд радиальной $V_r$ и угловой $V_\theta$ компонент:</p>

$$V_r =\frac{1}{i\rho c}p_0\sum\limits_{n=0}^{\infty}P_n(cos\theta)(2n+1)i^n\Bigl[j_n(kr) -\frac{j_n^{'}(ka)}{h_n^{'(1)}(ka)} h_n^{(1)}(kr)\Bigr] \tag{1.7}$$

$$V_\theta = \frac{1}{i\rho \omega}\frac{1}{r}p_0\sum\limits_{n=0}^{\infty} \frac{\partial{}}{\partial\theta} (P_n(cos\theta))(2n+1)i^n\Bigl[j_n(kr) -\frac{j_n^{'}(ka)}{h_n^{'(1)}(ka)} h_n^{(1)}(kr)\Bigr] \tag{1.8}$$

<p>Выражения (1.6), (1.7), (1.8) полностью описывают поле звуковой волны, рассеиваемой на абсолютно жесткой сфере радиуса $a$.</p>

<p>На основе описанной теории, приходим к выводу о том, что мы не можем реализовать бесконечное суммирование ряда (см. формулу $(1.2)$), поэтому нам нужно определить верхний предел суммирования, который бы смог гарантировать корректность всех рассчитываемых сумм. Таким образом, сначала решался вопрос поиска таких значений верхней границы суммирования $N_{max}$ данного ряда, для которых отбрасываемая часть суммы вносит незначительный вклад.</p>

<p>На помощь приходит представление или графическая интерпретация сферических функций Бесселя (рис. 1.2.), которые согласно формуле $(1.2)$ входят в искомый ряд.</p>

<img src=https://bit.ly/3b3lHYs width="500">

$$\textbf{Рис. 1.2.}~\text{Сферические функции Бесселя}$$

<p>Нетрудно заметить, что по мере увеличения порядка сферической функции, ее первый экстремум сдвигается вправо вдоль оси аргумента, а все, что находится левее первого экстремума, характеризуется малыми значениями. Соответственно, если зафиксировать произвольное значение аргумента, то наибольший вклад в искомый ряд будет вносить сферическая функция Бесселя наименьшего порядка, а каждая последующая функция  - все меньше и меньше, и, начиная с какого-то номера $N_{max}$, вклад функций с порядком большим, чем $N_{max}$, будет уже пренебрежимо мал для построения нашей модели.</p>
	
<p>Согласно условиям данной задачи нам требуется исследовать влияние падающего поля давления на рассеиватель, поэтому необходимо реализовать точный расчет лишь вблизи локализации рассеивателя.</p>
	
<p>Учитывая вышеизложенные факты, получаем, что при аргументе функции, которым является величина $kr$ согласно $(1.2)$, равном $N_{max}$, последующие сферические функции Бесселя порядков выше, чем $N_{max}$, не вносят существенный вклад в рассчитываемый ряд, а сам радиус области оказывается равным радиусу рассеивателя (назовем его $a$), умноженному на число из промежутка от трех до пяти.</p>
	
<p>Таким образом, мы получили выражение для предельных значений верхней границы суммирования, при которых расчитанное поле давлений сходится достаточно точно с теорией вблизи области $r = a$.</p>

$$N_{max} = (3\div5)ka. \tag{1.12}$$

<p>Получается, выбирая значение $N_{max}$, мы фактически задаем максимально возможные значения радиуса нашего рассеивателя. Если же радиус рассеивателя положить фиксированным, а значение верхней границы суммирования взять меньшим, чем значение, полученное в результате подстановки соответсвующего радиуса в уравнение $(1.12)$, мы можем столкнуться с неправильным подсчетом сферических функций в этой области.</p>

<p>В случае сферической падающей волны, согласно [6] существует похожее относительно $(1.2)$ разложение:</p>

$$P_s = \sum \limits_{n=0}^{\infty} A_{sp} ik(-1)^n(2n+1)h_n^{(1)}(kz_0)c_nh_n^{(1)}(kr)P_n(cos\theta), \tag{1.13}$$

<p>где $c_n$ определяется согласно формуле (1.3), а $h_n^{(1)}(kr) =j_n(kr) +iy_n(kr)$ - сферическая функция Ханкеля 1-го рода, $y_n(\zeta) = \sqrt{\pi/(2\zeta)}Y_{n+1/2}(\zeta)$, $Y_n(\zeta)$ - сферическая функция Неймана. Источник сферической волны находится в точке $z = z_0$.</p>

<p>В случае сферической падающей волны величина $P_{\text{пад}}$ представима в виде:</p>

$$P_{\text{пад}} = p_0e^{ikR}/R, \tag{1.14}$$

<p>где $R = \sqrt{r^2 + z_0^2+2rz_0cos\theta}~~$(рис. 1.5).</p>

<img src=https://bit.ly/2WpuPS5 width="500">

$$\textbf{Рис. 1.3.}~ \text{Геометрия задачи в случае сферической волны}$$

<h2>Рассмотрим теперь основную программу</h2>

<p>Для лучшей читаемости заменим $R$ из рис. 1.2 на функцию $\textit{sqrthcos}$, ее название связано с теоремой косинусов, применяемой для вычисления самой величины. На вход этой функции передаем радиус-вектор рассматриваемой точки, координату источника сферической волны, и косинус угла до радиус-вектора этой точки для однозначности. На выходе получаем значение $R$.</p>

<p>Инициализация начальных параметров, констант, вызов функции $\textit{upperRecurency}$ для расчета функций Бесселя и Неймана в точке с координатой источника волн, которые потом используются в формуле $(1.13)$</p>

In [289]:
# Square root + theorem of cos
def sqrthcos(r, z0, costheta):
    return (r ** 2 + z0 ** 2 + 2 * r * z0 * costheta) ** .5

# Initialize the parameters of required functions
# Nmax -> quantity of required functionsp
# z0 -> initial coordinate of the spherical-wave sourse
# step -> cells-step
Nmax = 10
z0 = 50
step = 1
a = 5 * (10 ** (-3))  # 5mm radius of sperical object
f = 10 ** 6  # 1 MHz
k = 2 * pi * f / 1500  # k = w / c, w = 2piv
# k = 1
p_0 = 1
A_sp = 1
rho = 1
# c_sound = 1 #1500
c_sound = 1500 #1500


# Initialize the matrix 401 x 401 ([-200, 200] x [-200, 200]) with complex data type for theoretical functions
p_ith = np.zeros((401, 401), dtype=complex)
p_spth = np.zeros((401, 401), dtype=complex)
p_sc = np.zeros((401, 401), dtype=complex)
p_sc_sp = np.zeros((401, 401), dtype=complex)


# Initialize the matrix 1 x Nmax for Legendre Polynomials
P = np.zeros(Nmax + 1)

# Initialize the matrix 401 x 401 ([-200, 200] x [-200, 200]) with complex data type for calculating functions
p_i = np.zeros((401, 401), dtype=complex)
p_sp = np.zeros((401, 401), dtype=complex)

# Calculating the Bessels functions with the help of our function called as 'upperRecurency' in the value z0
sphjz0, sphyz0 = upperRecurency(k * z0, Nmax)

t = time.time()
e = np.zeros((401, 401), dtype=float)
e_sp = np.zeros((401, 401), dtype=float)

<p>Для лучшего восприятия введем вспомогательные функции для сферического коэффициента $Q_n^{(spherical~wave)}$ и коэффициента плоской волны $Q_n^{(plane~wave)}$ согласно [6].</p>

<p>Формульно они записываются так:</p>

$$Q_n^{(spherical~wave)} = A_{sp} ik(-1)^n(2n+1)h_n^{(1)}(kz_0) \tag{1.15}$$

$$Q_n^{(plane~wave)} = p_0i^n(2n+1)\tag{1.16}$$

In [290]:
def Q_sp(n, sphjz0, sphyz0):
    return A_sp * 1j * k * ((-1) ** n) * (2 * n + 1) * (sphjz0[n] + 1j * sphyz0[n])

In [291]:
def Q_pl(n):
    return p_0 * (1j ** n) * (2 * n + 1)

<p>Вычислим коэффициенты $c_n$ в разложении $(1.13)$ используя $(1.5)$.</p>

<p>Радиационная сила, точнее ее проекция на ось $z$, может быть представлена как:</p>

$$F_z = \frac{2\pi}{\rho c^2 k^2}\sum_{n=0}^{\infty}\frac{(n+1)}{(2n+1)(2n+3)}Im[Q_nQ_{n+1}^*(c_n+c_{n+1}^*+2c_nc_{n+1}^*)]\tag{1.17}$$

<p>Таким образом, правильно вычислив сферические функции Бесселя и Неймана и их производные, мы можем вычислить коэффициенты $c_n$, с помощью которых получаем искомое выражения для проекции радиационной силы</p>

In [292]:
# calculate the coefficients in the sum for spherical wave
c = np.zeros(Nmax + 1, dtype=complex)
F_z_plane, F_z_spherical = 0, 0
sphja, sphya = upperRecurency(k * a, Nmax)
sphja_der, sphya_der = derivative(sphja, sphya, k * a)
# print(sphja_der,sphya_der)
for n in range(len(sphya)):
    c[n] = -(1j) ** n * (2 * n + 1) * sphja_der[n] / (sphja_der[n] + 1j * sphya_der[n]) 


for n in range(Nmax):
    F_z_spherical += 2 * pi / (rho * (c_sound ** 2) * (k ** 2)) *(n + 1) / ((2 * n + 1) * (2 * n + 3)) * np.imag(Q_sp(n, sphjz0, sphyz0) * np.conj(Q_sp(n + 1, sphjz0, sphyz0)) * (c[n] + np.conj(c[n + 1]) + 2 * c[n] * np.conj(c[n + 1])))
    F_z_plane += -2 * pi * (p_0 ** 2) / (rho * (c_sound ** 2) * (k ** 2)) * (n + 1) * np.real(c[n] + np.conj(c[n + 1]) + 2 * c[n] * np.conj(c[n + 1]))

<p>Посмотрим, чему равны значения рассчитанных радиационных сил, в ознакомительных целях</p>

In [293]:
print(f'F_z_spherical_wave = {F_z_spherical}')

F_z_spherical_wave = 3.0367272397429333e-13


In [294]:
print(f'F_z_plane_wave = {F_z_plane}')

F_z_plane_wave = 7.591665500338878e-10


<p>Следующий блок кода был положен в основу проверки корректности расчета сферических функций. С помощью формул $(1.2)$ и $(1.14)$ мы знаем математическое описание поля давлений в заданной области, поэтому, рассчитав частичные суммы рядов $(1.2)$ и $(1.13)$, мы можем изобразить модуль разности теоретических значений и рассчитанных, оценив, при этом, область сходимости, то есть ту область, в которой наши функции посчитаны верно.</p>
<p>Обратим внимание на полиномы Лежандра, которые встречаются в любой сумме, представленной выше (напр. $(1.2)$). Характерные графики полиномов Лежандра $P_n(cos\theta)$ в зависимости от угла $\theta$, изменяющегося от $0$ до $\pi$ представлены ниже:</p>

<img src='https://bit.ly/2SPCacE' width=500>

$$\textbf{рис. 1.4}~\text{Зависимость полиномов Лежандра}~P_n(cos\theta)~от~\theta$$

<p>При численном моделировании используем рекуррентное соотношение, позволяющее рассчитать любой заданный наперед порядок:</p>
$$P_{n+1}(cos\theta)=\frac{2n+1}{n+1}cos\theta \cdot P_n(cos\theta) - \frac{n}{n+1}P_{n-1}(cos\theta) \tag{1.18}$$

In [295]:
# Going through the cells
print('Percentage: ', '\r')
for x in range(-200, 201):
    print(x / 4 + 50, '\r', end='')
    for z in range(-200, 201):
        # Calculating the radius-vector in the current point
        r = step * (x ** 2 + z ** 2) ** .5
        
        # Check for non-zero value for r
        if r:
            
            # Calculating cos(theta) in the current point
            costheta = step * z / r
            
            R = sqrthcos(r, z0, costheta)
                
            # Calculating the Bessels functions with the help of our function called as 'upperRecurency' in the value r
            sphj, sphy = upperRecurency(k * r, Nmax)
                
            # Calculating the theoretical pressure functions
            p_ith[x + 200][z + 200] = p_0 * cmath.exp(1j * k * r * costheta)
                
            # Check for non-zero value for theorem of cos in the current point
            if sqrthcos(r, z0, costheta):
                p_spth[x + 200][z + 200] = p_0 * cmath.exp(1j * k * R) / R
                
            # Use bottom-to-top recurrency relation to calculate Legendre's polynomials
            P[0] = 1
            P[1] = costheta
            for n in range(1, Nmax):
                P[n + 1] = (2 * n + 1) / (n + 1) * costheta * P[n] - n / (n + 1) * P[n - 1];
                    
            # Calculating the required pressure functions (incident)
            for n in range(0, Nmax):
                p_i[x + 200, z + 200] += Q_pl(n) * P[n] * sphj[n]
                p_sp[x + 200, z + 200] += Q_sp(n, sphjz0, sphyz0) * P[n] * sphj[n] 
                
                #(scattered)
                p_sc [x + 200, z + 200] += Q_pl(n) * c[n] * (sphj[n] + 1j * sphy[n]) * P[n]
                p_sc_sp [x + 200, z + 200] += Q_sp(n, sphjz0, sphyz0) * c[n] * (sphj[n] + 1j * sphy[n]) * P[n]
                
            e[x + 200, z + 200] = abs(p_i[x + 200, z + 200] - p_ith[x + 200, z + 200])
            e_sp[x + 200, z + 200] = abs(p_sp[x + 200, z + 200] - p_spth[x + 200, z + 200])
                
print('time: ', time.time() - t)

Percentage:  
time:  57.616360902786255           


<p>Картинки, на которых можно оценить, как влияет $N_{max}$ на размеры области сходимости</p>

<img src='https://bit.ly/2WiSeWl' width=700>

$$\textbf{рис. 1.5.} ~~\text{$N_{max} = 10~$Плоская волна}$$

<p>На $\textbf{рис. 1.5}$ изображено распределение функции ошибки в случае падения на рассеиватель плоской волны при $N_{max} = 10$, то есть распределение модуля разности теоретического значения акустического давления в данной точке и посчитанного программой, см. $(1.2)$. Примечательным на этом графике является область нулевого значения функции ошибки, так называемая область сходимости, то есть та область (отмечена синим цветом, согласно цветовой палитре), в которой рассчитываемые функции полностью совпадают с теоретическими. Радиус области сходимости будет возрастать с увеличением верхнего индекса суммирования подобно тому, как учет последующего элемента в разложении ряда Тейлора все более точно описывает раскладываемую функцию.</p>

<img src='https://bit.ly/2SOL3mL' width=700>

$$\textbf{рис. 1.6.} ~~\text{$N_{max} = 10~$Сферическая волна}$$

<p>В случае же сферической волны, падающей на рассеиватель, мы получаем такую картину распределения функции ошибки. Примечательным здесь является то, что в месте, где находится источник сферической волны, наблюдается максимум ошибки, что оказывается следствием формулы $(1.14)$, в данной точке $R$ обращается в ноль, формируя деление на ноль в этой формуле. Следует отметить также малый радиус сходимости.</p>

<p>На последующих рисунках мы можем пронаблюдать явное изменение радиуса сходимости в зависимости от $N_{max}$</p>

<img src='https://bit.ly/3bfrSZG' width=700>

$$\textbf{рис. 1.7.} ~~\text{$N_{max} = 30~$Плоская волна}$$

<img src='https://bit.ly/2SRnSs4' width=700>

$$\textbf{рис. 1.8.} ~~\text{$N_{max} = 30~$Сферическая волна}$$

In [296]:
x = np.arange(-200, 201, 1)
y = np.arange(-200, 201, 1)

z = np.zeros((401, 401), dtype=float)
for ind in range(len(e)):
    for sec_ind in range(len(e[ind])):
        z[ind][sec_ind] = e[ind][sec_ind]
        
z2 = np.zeros((401, 401), dtype=float)
for ind in range(len(e_sp)):
    for sec_ind in range(len(e_sp[ind])):
        z2[ind][sec_ind] = e_sp[ind][sec_ind]


for ind in range(len(z)):
    for sec_ind in range(len(z[ind])):
#         print(z[ind][sec_ind] - e[ind][sec_ind])
        if z[ind][sec_ind] > 2:
#             print(p_i[ind][sec_ind], p_ith[ind][sec_ind])
            z[ind][sec_ind] = 2
    
for ind in range(len(z2)):
    for sec_ind in range(len(z2[ind])):
#         print(z[ind][sec_ind] - e[ind][sec_ind])
        if z2[ind][sec_ind] > .1:
#             print(p_i[ind][sec_ind], p_ith[ind][sec_ind])
            z2[ind][sec_ind] = .1

In [297]:
# fig = go.Figure(data=[go.Surface(z=z, x=x, y=y)])
# fig.update_layout(title='Error Function Plane Wave', autosize=False,
#                   width=700, height=700,
#                   margin=dict(l=65, r=50, b=65, t=90))
# fig.show()

In [298]:
# fig = go.Figure(data=[go.Surface(z=z2, x=x, y=y)])
# fig.update_layout(title='Error Function Spherical Wave', autosize=False,
#                   width=700, height=700,
#                   margin=dict(l=65, r=50, b=65, t=90))
# fig.show()

<ul>Список Литературы:</ul>
<p>1) $\textit{Hasegawa T., Yosioka K.}$ Acoustic-radiation force on a solid elastic sphere // J. Acoust. Soc. Amer. 1969. \\ V. 46. №. 5B. P. 1139 - 1143.</p>
<p>2) $\textit{Морс Ф. М., Фешбах Г.}$ Методы теоретической физики. М.: ИЛ, 1960. Т. 2.</p>
<p>3) $\textit{Lee C. P., Wang T. G.}$ Acoustic radiation pressure // J. Acoust. Soc. Amer. 1993. V. 94. No.
 2. P. 1099 – 1109.</p>
<p>4) $\textit{Hasegawa T. Kido T., Iizuka T., Matsuoka C.}$ A general theory of Rayleigh and Langevin radiation pressures // Acoust. Science Techn. 2001. V. 21. No. 3. P. 145 – 152.</p>
<p>5) $\textit{Ландау Л.Д., Лифшиц Е.М.}$ Гидродинамика, Ч. 1. M.: Наука, 1987, Глава 1. Идеальные среды. С. 1 – 43.</p>
<p>6) $\textit{Sapozhnikov O. A., Bailey M. R.}$ Radiation force on an elastic sphere //J. Acoust. Soc. Am. 2013  Vol. 133. №. 2. P. 661 - 676.</p>
<p>7)$~$<a>https://en.wikipedia.org/wiki/Bessel_function#Spherical_Bessel_functions</a></p>
<p>8)$~$<a>https://docs.scipy.org/doc/scipy/reference/special.html</a></p>
<p>9)$~$<a>https://plotly.com/python/</a></p>